<a href="https://colab.research.google.com/github/mjsipes/stock_algorithms/blob/main/StockAlgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
def perfectalgo(data, initial_investment, fee_percent, DEBUG):
    current_cash = initial_investment
    stocks_owned = 0
    fee_decimal = fee_percent / 100.0

    price = [0] * len(data)
    difference = [0] * len(data)
    pdifference = [0] * len(data)
    direction = ["-"] * len(data)
    action = ["HOLD"] * len(data)
    networth = [0] * len(data)

    for i in range(len(data)):
        price[i] = data["Close"].iloc[i]
        if i > 0:
            difference[i] = data["Close"].iloc[i] - data["Close"].iloc[i-1]
            pdifference[i] = (data["Close"].iloc[i] - data["Close"].iloc[i-1]) / data["Close"].iloc[i]
            direction[i] = "UP" if (data["Close"].iloc[i] - data["Close"].iloc[i-1]) > 0 else "DOWN"

            if i < len(data)-1 and data["Close"].iloc[i+1] > data["Close"].iloc[i]:
                if current_cash > 0:
                    action[i] = "BUY"
                    stocks_to_buy = current_cash / (data["Close"].iloc[i] * (1 + fee_decimal))
                    stocks_owned += stocks_to_buy
                    current_cash = 0

            elif i < len(data)-1 and data["Close"].iloc[i+1] < data["Close"].iloc[i]:
                if stocks_owned > 0:
                    action[i] = "SELL"
                    cash_from_sale = stocks_owned * data["Close"].iloc[i] * (1 - fee_decimal)
                    total_profit = cash_from_sale - initial_investment
                    current_cash = cash_from_sale
                    stocks_owned = 0
            networth[i] = current_cash + stocks_owned * data["Close"].iloc[i]

    if DEBUG:
        print(f'{"Index":<10}{"Price":<10}{"Difference":<15}{"%Difference":<15}{"Direction":<10}{"Action":<10}{"Networth":<10}')
        for i in range(len(data)):
            print(f'{i:<10}{price[i]:<10.2f}{difference[i]:<15.2f}{"{:.2f}%".format(pdifference[i]*100):<15}{direction[i]:<10}{action[i]:<10}{networth[i]:<10.2f}')

    return current_cash + stocks_owned * data["Close"].iloc[-1]


In [30]:
def greedyalgo(data, initial_investment, fee_percent, DEBUG):
    current_cash = initial_investment
    stocks_owned = 0
    fee_decimal = fee_percent / 100.0

    price = [0] * len(data)
    difference = [0] * len(data)
    pdifference = [0] * len(data)
    direction = ["-"] * len(data)
    action = ["HOLD"] * len(data)
    networth = [0] * len(data)

    for i in range(len(data)):
        price[i] = data["Close"].iloc[i]
        if i > 1:
            difference[i] = data["Close"].iloc[i] - data["Close"].iloc[i-1]
            pdifference[i] = (data["Close"].iloc[i] - data["Close"].iloc[i-1]) / data["Close"].iloc[i]
            direction[i] = "UP" if (data["Close"].iloc[i] - data["Close"].iloc[i-1]) > 0 else "DOWN"

            if data["Close"].iloc[i] > data["Close"].iloc[i - 1] and data["Close"].iloc[i-1] < data["Close"].iloc[i - 2]:
                if current_cash > 0:
                    action[i] = "BUY"
                    stocks_to_buy = current_cash / (data["Close"].iloc[i] * (1 + fee_decimal))
                    stocks_owned += stocks_to_buy
                    current_cash = 0

            elif data["Close"].iloc[i] < data["Close"].iloc[i - 1] and data["Close"].iloc[i-1] > data["Close"].iloc[i - 2]:
                if stocks_owned > 0:
                    action[i] = "SELL"
                    cash_from_sale = stocks_owned * data["Close"].iloc[i] * (1 - fee_decimal)
                    total_profit = cash_from_sale - initial_investment
                    current_cash = cash_from_sale
                    stocks_owned = 0
            networth[i] = current_cash + stocks_owned * data["Close"].iloc[i]

    if DEBUG:
        print(f'{"Index":<10}{"Price":<10}{"Difference":<15}{"%Difference":<15}{"Direction":<10}{"Action":<10}{"Networth":<10}')
        for i in range(len(data)):
            print(f'{i:<10}{price[i]:<10.2f}{difference[i]:<15.2f}{"{:.2f}%".format(pdifference[i]*100):<15}{direction[i]:<10}{action[i]:<10}{networth[i]:<10.2f}')

    return current_cash + stocks_owned * data["Close"].iloc[-1]


In [31]:
import sys
import os
import yfinance as yf
from datetime import datetime, timedelta

ticker_symbol = "DOGE-USD"
initial_investment = 100
interval = "1m"
fee_percent = 0.1
DEBUG = True

# Get today's date
today = datetime.now()

# Check if today is Saturday or Sunday, and adjust to the most recent Friday
if today.weekday() >= 5:  # 5: Saturday, 6: Sunday
    days_to_subtract = today.weekday() - 4  # Adjust to Friday
    recent_trading_day = today - timedelta(days=days_to_subtract)
else:
    recent_trading_day = today

year = recent_trading_day.year
month = recent_trading_day.month
day = recent_trading_day.day

# Suppress yfinance download output
original_stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')
data = yf.download(ticker_symbol, start=f"{year}-{month:02d}-{day:02d}", end=f"{year}-{month:02d}-{day+1:02d}", interval=interval)
sys.stderr = original_stderr

# Calculate profits using the algorithms
# perfect_algo_profit = perfectAlgo(data, initial_investment, fee_percent, DEBUG)
greedy_algo_profit = greedyalgo(data, initial_investment, fee_percent, DEBUG)


Index     Price     Difference     %Difference    Direction Action    Networth  
0         0.12      0.00           0.00%          -         HOLD      0.00      
1         0.12      0.00           0.00%          -         HOLD      0.00      
2         0.12      0.00           0.01%          UP        BUY       99.90     
3         0.12      0.00           0.00%          UP        HOLD      99.90     
4         0.12      0.00           0.01%          UP        HOLD      99.92     
5         0.12      0.00           0.05%          UP        HOLD      99.97     
6         0.12      0.00           0.18%          UP        HOLD      100.15    
7         0.12      -0.00          -0.02%         DOWN      SELL      100.03    
8         0.12      -0.00          -0.01%         DOWN      HOLD      100.03    
9         0.12      -0.00          -0.04%         DOWN      HOLD      100.03    
10        0.12      0.00           0.03%          UP        BUY       99.93     
11        0.12      0.00    

In [32]:
from datetime import datetime
def is_weekend(year, month, day):
    date_obj = datetime(year, month, day)
    return date_obj.weekday() >= 5



In [33]:
import sys
import os
import yfinance as yf

def analyze_day(year, month, day, ticker_symbol, initial_investment, interval, fee_percent, DEBUG):
  try:
    original_stderr = sys.stderr
    sys.stderr = open(os.devnull, 'w')
    data = yf.download(ticker_symbol, start=f"{year}-{month:02d}-{day:02d}", end=f"{year}-{month:02d}-{day+1:02d}", interval=interval)
    sys.stderr = original_stderr

    greedy_algo_profit = greedyalgo(data, initial_investment,fee_percent, DEBUG)
    perfect_algo_profit = perfectalgo(data, initial_investment,fee_percent, DEBUG)

    return greedy_algo_profit, perfect_algo_profit
  except Exception as e:
    print(f"Error processing {ticker_symbol} on {year}-{month}-{day}: {e}")
    return 0, 0  # Return 0 profits if there's an error


In [34]:
def format_profit_list(profit_list):
    return ' '.join(f'{profit:8.2f}' for profit in profit_list)


In [35]:

import calendar

# Get the current date
current_date = datetime.now()

# Calculate the start date (30 days ago)
start_date = current_date - timedelta(days=30)

ticker_symbol_list = ["AAPL", "GOOG", "MSFT", "IBM", "NVDA", "RNG", "BTC-USD", "ETH-USD", "DOGE-USD", "SHIB-USD", "ADA-USD"]
initial_investment = 100
interval = "5m"
fee_percent = 0  # Binance maker-taker fee is 0.1%
DEBUG = False

for ticker_symbol in ticker_symbol_list:
    greedyprofit = []
    perfectprofit = []

    # Iterate from 30 days ago until today
    for n in range(30):
        day_date = start_date + timedelta(days=n)
        year = day_date.year
        month = day_date.month
        day = day_date.day

        # Check if the day is within the valid range of the month
        if day <= calendar.monthrange(year, month)[1] and not is_weekend(year, month, day):
            cgreedyprofit, cperfectprofit = analyze_day(year, month, day, ticker_symbol, initial_investment, interval, fee_percent, DEBUG)
            greedyprofit.append(cgreedyprofit)
            perfectprofit.append(cperfectprofit)

    # Calculate and print the average profits
    if greedyprofit and perfectprofit:  # Ensure there are no division errors
        average_daily_greedy_profit = sum(greedyprofit) / len(greedyprofit)
        average_daily_perfect_profit = sum(perfectprofit) / len(perfectprofit)
        print(f"{ticker_symbol} avg daily perfect profit: {average_daily_perfect_profit:.2f}")
        print(f"{ticker_symbol} avg daily greedy profit:  {average_daily_greedy_profit:.2f}")
        print(f"{ticker_symbol} daily perfect profit:     {format_profit_list(perfectprofit)}")
        print(f"{ticker_symbol} daily greedy profit:      {format_profit_list(greedyprofit)}")
        print()



ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: ValueError('unconverted data remains: 2')


Error processing AAPL on 2024-7-31: single positional indexer is out-of-bounds
AAPL avg daily perfect profit: 99.22
AAPL avg daily greedy profit:  94.99
AAPL daily perfect profit:       103.66   104.02   103.27   103.77   104.96   103.79   103.50   102.93   103.63   106.09   104.33   103.22   103.28   103.42   105.62   104.89   103.45   103.83     0.00   104.53   107.44
AAPL daily greedy profit:        100.30   101.11    99.56    99.83    99.35    99.42    99.69    99.02   100.35    99.59    99.08    98.62   100.36    98.27   101.81    99.13   100.05    99.97     0.00    98.48   100.69



ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GOOG']: ValueError('unconverted data remains: 2')


Error processing GOOG on 2024-7-31: single positional indexer is out-of-bounds
GOOG avg daily perfect profit: 99.01
GOOG avg daily greedy profit:  94.72
GOOG daily perfect profit:       103.24   102.75   102.11   102.77   102.89   103.71   103.75   102.64   102.84   104.88   103.60   104.45   103.73   105.99   106.61   106.21   104.09   103.53     0.00   104.77   104.70
GOOG daily greedy profit:        100.08   100.16    99.85   100.08    98.47   100.10    98.99    98.98    99.08    98.83    98.78    98.85    99.85    99.08   101.18    99.48    99.75    99.62     0.00    98.31    99.52



ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MSFT']: ValueError('unconverted data remains: 2')


...

Error processing DOGE-USD on 2024-7-31: single positional indexer is out-of-bounds
DOGE-USD avg daily perfect profit: 120.48
DOGE-USD avg daily greedy profit:  99.46
DOGE-USD daily perfect profit:       155.07   145.02   120.76   118.80   118.64   116.61   130.31   130.65   122.00   121.04   124.79   134.62   126.66   121.54   127.32   123.83   120.25   119.36     0.00   126.34   126.56
DOGE-USD daily greedy profit:        110.03   105.91   104.35   104.09   103.69   104.10   108.36   105.05   100.39   103.49   106.67   107.56    98.31   102.89   105.66   104.85   103.29   103.48     0.00   105.34   101.09



ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SHIB-USD']: ValueError('unconverted data remains: 2')


Error processing SHIB-USD on 2024-7-31: single positional indexer is out-of-bounds
SHIB-USD avg daily perfect profit: 122.79
SHIB-USD avg daily greedy profit:  99.35
SHIB-USD daily perfect profit:       165.92   162.68   128.38   124.21   121.95   121.99   133.25   142.61   126.31   138.19   132.20   120.03   122.66   117.39   122.32   117.30   116.59   115.18     0.00   125.40   124.01
SHIB-USD daily greedy profit:        111.77   107.14   103.68   105.85   103.03   105.85   109.01   107.04    96.75   102.03   107.11   101.25   101.82   102.65   105.61   103.64   103.62   103.01     0.00   103.76   101.84



ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ADA-USD']: ValueError('unconverted data remains: 2')


Error processing ADA-USD on 2024-7-31: single positional indexer is out-of-bounds
ADA-USD avg daily perfect profit: 115.22
ADA-USD avg daily greedy profit:  100.70
ADA-USD daily perfect profit:       140.99   142.85   118.65   116.91   120.75   123.61   118.18   122.63   118.83   115.31   120.41   114.32   116.30   117.25   118.64   117.06   116.31   115.06     0.00   123.60   121.89
ADA-USD daily greedy profit:        112.29   110.57   102.68   106.60   106.30   108.41   105.92   106.29   107.06   104.06   107.07   101.47   101.46   104.09   106.04   105.81   105.72   103.45     0.00   106.08   103.23

